# Cube Runner
- to play, Run all Cells and the game window will pop up
- this game uses pyglet, which may have to be installed
- version for github


In [309]:
from pyglet.gl import *
from pyglet.window import *
from pyglet.media import *
from pyglet.text import Label
from pyglet.app import EventLoop
import numpy as np
import random

# colors

In [310]:
white = (1,1,1)
black = (0,0,0)
red = (1,0,0)
green = (0,0.75,0)
blue = (0,0.5,1)
orange = (1,0.5,0)
carolina = (0.6,0.8,1)
neon_yellow = (0.9,1,0.1)
dark_green = (0.4,0.6,0.2)
brown = (0.5,0.2,0)
tan = (0.9,0.9,0.6)

# 2D and 3D set
- these functions set up two-dimensional and three-dimensional views
- both are used in the game

In [311]:
def set2D(window):
    '''sets up matrix for two-dimensional view and usage'''
    window.clear()
    glDisable(GL_DEPTH_TEST)
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    glOrtho(0.0, window.width, 0.0, window.height, 0.0, 1.0)    # 2D perspective

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
def set3D(window):
    '''sets up matrix for three-dimensional view and usage'''
    window.clear()
    glEnable(GL_DEPTH_TEST)         # enable depth testing
    
    glMatrixMode(GL_PROJECTION)         # reset matrices
    glLoadIdentity()                    # load the identity matrix

    gluPerspective(45, window.width/window.height, 0.1, 500)    # 3D perspective
    
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    glTranslatef(0,0,-15)
    glRotatef(10,1,0,0)                  # starting view

# game object classes
- these classes represent the different objects which appear in the game
- the level tracker
- the cubes which are responsible for both:
    - the user
    - the obstacles

# level 

In [312]:
class Level():
    '''object to hold level characteristics'''

    def __init__(self):
        self.level=0
        self.speed=0.2              # how fast the obstacles come
        self.speed_increase=0.025
        self.length=1200           # until obstacles stop getting created
        self.timer=0
        self.over=False       # when creation has stopped
        self.end_timer=0
        self.end_length=300    # time between end of obstacles and end of level
        self.surface_color = ('c3f', red*4)
        self.edge_color = ('c3f',black*2)

# cube

In [313]:
class Cube():
    '''cubes for obstacles and user'''
    
    def __init__(self,x,y,z,s, color, edge_color):
        self.surface_batch = pyglet.graphics.Batch()    # batched rendering
        self.edge_batch = pyglet.graphics.Batch()
        
        self.surface_verts = []    # vertices needed for movement / collision detection
        self.edge_verts = []
        
        X,Y,Z = x+s,y+s,z+s       # coordinate creation
        surface_coords = np.array([
            [X, y, z,  x, y, z,  x, Y, z,  X, Y, z], #back
            [x, y, Z,  X, y, Z,  X, Y, Z,  x, Y, Z], #front
            
            [x, y, z,  x, y, Z,  x, Y, Z,  x, Y, z], #left
            [X, y, Z,  X, y, z,  X, Y, z,  X, Y, Z], #right
            
            [x, y, z,  X, y, z,  X, y, Z,  x, y, Z], #bottom
            [x, Y, Z,  X, Y, Z,  X, Y, z,  x, Y, z]  #top
        ])
        
        for coord in surface_coords:
            self.surface_verts.append(self.surface_batch.add(4, GL_QUADS, None, ('v3f', coord), color))
            
        edge_coords = np.array([
            [X, y, z,  x, y, z],  # around back face
            [x, y, z,  x, Y, z],
            [x, Y, z,  X, Y, z],
            [X, Y, z,  X, y, z],
            
            [x, y, Z,  X, y, Z],   # around front face
            [X, y, Z,  X, Y, Z],
            [X, Y, Z,  x, Y, Z],
            [x, Y, Z,  x, y, Z],
            
            [X, y, z,  X, y, Z],   # connecting back & front faces
            [x, y, z,  x, y, Z],
            [x, Y, z,  x, Y, Z],
            [X, Y, z,  X, Y, Z]
        ])
        
        for coord in edge_coords:
            self.edge_verts.append(self.edge_batch.add(2, GL_LINES, None, ('v3f', coord), edge_color))
        
    def draw(self): 
        self.surface_batch.draw()
        self.edge_batch.draw()

# obstacles

In [314]:
class Obstacles():
    '''holds list of all the current obstacles'''
    
    def __init__(self,h,color,edge_color):
        self.color=color
        self.edge_color=edge_color
        self.size=0.7
        self.separation=1.5
        self.offset=0.75
        self.distance=2.5
        self.x=9
        self.dx=-self.x
        self.cubes=[]
        self.is_right=False
        
        self.z = -10
        num = 20
        funnel_sep = (self.x-self.distance)/num
        for i in range(num):
            self.x -= funnel_sep
            self.dx += funnel_sep
            self.z -= self.separation
            self.cubes.append( Cube(self.x, h, self.z, self.size, color, edge_color) ) 
            self.cubes.append( Cube(self.dx, h, self.z, self.size, color, edge_color) )
            
    def draw(self): 
        for cube in reversed(self.cubes): cube.draw()     # reversed so cubes appear in the correct order
            
    def update(self,user):
        '''updates list to remove passed obstacles and replace with new ones'''
        if self.cubes[0].surface_verts[0].vertices[-1] > user.surface_verts[0].vertices[-1]+10:
            self.cubes[:-2] = self.cubes[2:]     # removes the first two cubes from list
            
            self.is_right = random.choice([True,False]+[self.is_right]*4)    # creates the changing pathway
            if self.is_right: 
                self.x += self.offset
                self.dx += self.offset
            else: 
                self.x -= self.offset
                self.dx -= self.offset
                
            self.z -= self.separation
            self.cubes[-2] = Cube(self.x, -3, self.z, self.size, self.color, self.edge_color)
            self.cubes[-1] = Cube(self.dx, -3, self.z, self.size, self.color, self.edge_color)

# user

In [315]:
class User(Cube):
    '''cube controlled by user during play'''
    
    def move_horizontal(self,dx,obst):
        '''controls horizontal movement based on arrow key presses'''
        speed_factor = obst.separation / obst.offset
        dx /= speed_factor             # factor of speed
        for element in self.surface_verts:
            for i in range(0,10,3):
                element.vertices[i]+=dx
        for element in self.edge_verts:
            element.vertices[0]+=dx
            element.vertices[3]+=dx
        glTranslatef(-dx,0,0)
            
    def advance(self,dz):
        '''controls forward movement'''
        for element in self.surface_verts:
            for i in range(2,12,3):
                element.vertices[i]+=dz
        for element in self.edge_verts:
            element.vertices[2]+=dz
            element.vertices[5]+=dz
        glTranslatef(0,0,-dz)

# game flow classes
- these are classes which represent each section of the game
- they include messages, updates, and anything necessary to display for that part of the game
- the different game sections are:
    - the introduction screen
    - the instructions screen
    - the level up screen
    - the game play screen
    - the game over screen
    
- the game play screen is the main section, it is the only 3D area where the game is actually played
- the rest of the sections are just menus / word displays

# parent Menu class
- super methods for intro, instructions, game over

In [316]:
class Menu():
    def check_cursor(self,x,y,window, x1,x2,y1,y2, *args):
        '''detect user mouse over buttons to change cursor'''
        if (x1 < x < x2 and y1 < y < y2) or (len(args)>0 and args[0] < x < args[1] and args[2] < y < args[3]):
            window.set_mouse_cursor(window.get_system_mouse_cursor(window.CURSOR_HAND))
        else:
            window.set_mouse_cursor(window.get_system_mouse_cursor(window.CURSOR_DEFAULT))
    
    def check_click(self,x,y,button,window,event_loop, x1,x2,y1,y2, intro=None, **kwargs):
        '''detect click on buttons'''
        if x1 < x < x2 and y1 < y < y2:
            if button == pyglet.window.mouse.LEFT:
                window.set_mouse_cursor(window.get_system_mouse_cursor(window.CURSOR_DEFAULT))
                event_loop.exit()
                if 'start' in kwargs:            # starting game
                    level_up(Level(), window)
                elif 'instr' in kwargs:          # loading instructions screen
                    instructions(window,intro)
                else:                            # back to intro screen
                    intro_loop(window,intro)

# Intro

In [317]:
class Intro(Menu):
    def __init__(self,window):
        self.contents = [
            Label('Cube Runner', 
                   font_name='Dead Kansas', font_size=70,
                   x=window.width/2, y=window.height/1.75,
                   anchor_x='center', anchor_y='center'),
            Label('Play', 
                   font_name='Dead Kansas', font_size=45,
                   x=window.width/2, y=window.height/3,
                   anchor_x='center', anchor_y='center'),
            Label('Instructions', 
                   font_name='Dead Kansas', font_size=30,
                   x=window.width/2, y=window.height/5,
                   anchor_x='center', anchor_y='center')
        ]
        
    def check_cursor(self,x,y,window):
        super().check_cursor(x,y,window, 475,625,180,250, 400,700,100,160)
        
    def check_click(self,x,y,button,window,event_loop,*args):
        super().check_click(x,y,button,window,event_loop, 475,625,180,250, start=True)
        super().check_click(x,y,button,window,event_loop, 400,700,100,160, self, instr=True)           

# Instructions

In [318]:
class Instructions(Menu):
    def __init__(self,window):
        self.contents = [
            Label('''How To Play: 
                     Use the left and right arrow keys to move
                     Don\'t hit the cubes!''', 
                     font_name='Dead Kansas', font_size=30,
                     multiline=True, width=400,
                     x=10, y=window.height-10,
                     anchor_x='left', anchor_y='top'),
            Label('OKAY', 
                   font_name='Dead Kansas', font_size=30,
                   x=10, y=20,
                   anchor_x='left', anchor_y='baseline')
        ]
        
    def check_cursor(self,x,y,window):
        super().check_cursor(x,y,window, 10,125,20,75)
            
    def check_click(self,x,y,button,window,event_loop,intro):
        super().check_click(x,y,button,window,event_loop, 10,125,20,75,intro)

# LevelUp

In [319]:
class LevelUp():
    def __init__(self,level,window):
        self.update(level)
        self.timer = 0
        self.contents = [
            Label('Level '+str(level.level), 
                   font_name='Dead Kansas', font_size=50,
                   x=window.width/2, y=window.height/2,
                   anchor_x='center', anchor_y='center')
        ]
        
        
    def color(self,surface,edge,level):
        '''put color in correct format for GL'''
        level.surface_color = ('c3f',surface)
        level.edge_color = ('c3f',edge)
    
    def update(self,level):
        '''updates including color scheme based on level'''
        level.level += 1
        level.speed += level.speed_increase
        
        if level.level % 10 == 1:
            self.color(red*4, black*2, level)
        elif level.level % 10 == 2:
            self.color((1,1,1, 0.8,0.7,0.2, 0.8,0.7,0.2, 1,1,1), (0,0.5,1, 0,0.5,1), level)  #bluegold
        elif level.level % 10 == 3:
            self.color(neon_yellow*4, neon_yellow*2, level)
        elif level.level % 10 == 4:
            self.color((0.8,0,0, 1,0.7,0, 1,0.7,0, 0.8,0,0), (0.8,0,0, 1,0.7,0), level)   #redorange
        elif level.level % 10 == 5:
            self.color((0,0.5,1, 1,1,1, 1,0,0, 1,1,1), (0,0.5,1, 1,0,0), level)  #redwhiteblue
        elif level.level % 10 == 6:
            self.color((1,1,1, 1,1,1, 0,0,0, 0,0,0), (1,1,1, 0,0,0), level)    #blackwhite
        elif level.level % 10 == 7:
            self.color((1,0.5,0, 0,0.3,1, 0,0,1, 1,0,1), white*2, level)
        elif level.level % 10 == 8:
            self.color(carolina*4, black*2, level)
        elif level.level % 10 == 9:
            self.color(orange*4, white*2, level)
        elif level.level % 10 == 0:
            self.color(tan*4, tan*2, level)
            
    def time(self,level,window):
        '''determine time until next level starts'''
        self.timer += 1
        if self.timer < 100:
            window.dispatch_events()
            window.dispatch_event('on_draw')
            window.flip()
        else:
            game_play(level,window)

# GamePlay

In [320]:
class GamePlay():
    def __init__(self,level,window):
        self.user = User(0,-3,-1.5, 0.45, ('c3f',red*4), ('c3f',red*2))
        self.obstacles = Obstacles(-3, level.surface_color, level.edge_color)
        self.keys = {}
        self.contents = [self.user,self.obstacles]
        
    def set_keys(self,symbol):
        '''flag key when pressed'''
        self.keys[symbol] = True
        
    def check_keys(self,symbol):
        '''unflag key when released'''
        if self.keys[symbol]: 
            self.keys[symbol] = not self.keys[symbol]
    
    def update(self,level,window):
        '''update obstacles and timer'''
        self.user.advance(-level.speed)
        
        if level.timer < level.length:      
            level.timer += 1
            self.obstacles.update(self.user)  
        else:
            level.over = True
            
        if level.over:
            if level.end_timer < level.end_length: 
                level.end_timer += 1
            else: 
                level.end_timer = 0
                level.over = False
                level.timer = 0
                level_up(level,window)
                
        self.check_key_hold(level)
        self.check_collision(level,window)
        self.update_window(window)
                
    def check_key_hold(self,level):
        '''check if user is holding arrow keys to move'''
        for symbol in self.keys:
            if self.keys[symbol]:
                if symbol == key.RIGHT:
                    self.user.move_horizontal(level.speed,self.obstacles)
                if symbol == key.LEFT:
                    self.user.move_horizontal(-level.speed,self.obstacles)
                    
    def check_collision(self,level,window):
        '''check for overlapping vertices indicating a collision'''
        for cube in self.obstacles.cubes:
            x1= cube.surface_verts[0].vertices[3]
            x2= cube.surface_verts[0].vertices[0]
            z1= cube.surface_verts[0].vertices[-1]
            z2= cube.surface_verts[1].vertices[-1]
            ux1= self.user.surface_verts[0].vertices[3]
            ux2= self.user.surface_verts[0].vertices[0]
            uz1= self.user.surface_verts[0].vertices[-1]
            uz2= self.user.surface_verts[1].vertices[-1]
            if x1 < ux1 < x2 or x1 < ux2 < x2:
                if z1 < uz1 < z2 or z1 < uz2 < z2:  
                    game_over(level,window) 
                    
    def update_window(self,window):
        '''refresh screen manually'''
        window.dispatch_events()
        window.dispatch_event('on_draw')
        window.flip()

# GameOver

In [321]:
class GameOver(Menu):
    def __init__(self,level,window):
        self.contents = [
            Label('GAME OVER', 
                   font_name='Dead Kansas', font_size=70,
                   x=window.width/2, y=window.height/1.5,
                   anchor_x='center', anchor_y='center'),
            Label('Level '+str(level.level), 
                   font_name='Dead Kansas', font_size=35,
                   x=window.width/2, y=window.height/2.25,
                   anchor_x='center', anchor_y='center'),
            Label('Home', 
                   font_name='Dead Kansas', font_size=30,
                   x=window.width/2, y=window.height/5.5,
                   anchor_x='center', anchor_y='center')
        ]
        
    def check_cursor(self,x,y,window):
        super().check_cursor(x,y,window, 500,600,90,140)
            
    def check_click(self,x,y,button,window,event_loop,*args):
        super().check_click(x,y,button,window,event_loop, 500,600,90,140)

# game/menu loops
- these functions control the loops for each section of the game
- included in each function are the event handlers
    - these methods handle all events relevant to that section
    - each event handling method is decorated as such to alert pyglet 
- each function represents a loop
    - both custom loops and pyglet built-in event loops are utilized depending on scenario

# event handler class
- provides event handlers for the majority of the game

In [322]:
class EventHandler():
    def __init__(self,obj,window,event_loop=None,intro=None):
        self.obj = obj
        self.intro = intro
        self.window = window
        self.event_loop = event_loop
        
    def on_mouse_motion(self,x,y,dx,dy):
        self.obj.check_cursor(x,y,self.window)
    def on_mouse_press(self,x,y,button,modifiers): 
        self.obj.check_click(x,y,button,self.window,self.event_loop,self.intro)
    def on_draw(self): 
        draw_contents(self.window,self.obj)
    def on_close(self): 
        close_window(self.window,self.event_loop)

In [323]:
# additional methods

def draw_contents(window,obj):
    '''update text on screen'''
    window.clear()
    for thing in obj.contents: thing.draw()
        
def close_window(window,event_loop=None):
    '''exit the event loop and close the window'''
    if event_loop is not None: event_loop.exit()
    window.close()

# intro


In [324]:
def intro_loop(window,intro=None):
    if intro is None: 
        intro = Intro(window)
        
    event_loop=EventLoop()
    event_handler = EventHandler(intro,window,event_loop)
    window.set_handlers(event_handler)
    
    window.dispatch_event('on_draw')
    window.flip()
    
    event_loop.run()

# instructions

In [325]:
def instructions(window,intro):
    instr = Instructions(window)
    
    event_loop=EventLoop()
    event_handler = EventHandler(instr,window,event_loop,intro)
    window.set_handlers(event_handler)
    
    window.dispatch_event('on_draw')
    window.flip()
    
    event_loop.run()

# level up
- should show before each level

In [326]:
def level_up(level,window):
    set2D(window); lvl=LevelUp(level,window)
    
    event_handler = EventHandler(lvl,window)
    window.set_handlers(event_handler.on_draw, event_handler.on_close)
    
    while True: lvl.time(level,window)

# game play
- includes additional event handlers not needed anywhere else

In [327]:
def game_play(level,window):
    set3D(window); gm=GamePlay(level,window)
    
    event_handler = EventHandler(gm,window)
    window.set_handlers(event_handler.on_draw, event_handler.on_close)
    
    @window.event
    def on_key_press(symbol,modifiers): gm.set_keys(symbol)
    @window.event
    def on_key_release(symbol,modifiers): gm.check_keys(symbol)
    
    while True: gm.update(level,window)  

# game over

In [328]:
def game_over(level,window):
    set2D(window); gmovr=GameOver(level,window)
    
    event_loop=pyglet.app.EventLoop()
    event_handler = EventHandler(gmovr,window,event_loop)
    window.set_handlers(event_handler)
    
    event_loop.run()

# main
- calling main initializes the window and then calls for the intro screen to appear
- from there, the rest of the game is played

In [329]:
def main():
    window = Window(width=1100, height=650, caption='Cube Runner')    # pyglet window display
    window.set_location(90,50)                                       # fixed location on screen
    set2D(window)

    intro_loop(window)

In [330]:
main()

AttributeError: 'NoneType' object has no attribute 'flip'